<a href="https://colab.research.google.com/github/dev-sampsonorson/chibuzor/blob/main/assignment-1/Assignment_1_Part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Step 1: Load data into Google Colab

In [1]:
import pandas as pd
from tabulate import tabulate

import seaborn as sn
import matplotlib.pyplot as plt

from sklearn.feature_selection import VarianceThreshold

url_csv = 'https://raw.githubusercontent.com/dev-sampsonorson/chibuzor/main/assignment-1/Part1.csv'

df = pd.read_csv(url_csv)

### Step 2: Explore the dataset
**Display the dataframe**

In [ ]:
# top 5, bottom 5
df

,ID,Insurance Type,Income of Policy Holder,Marital Status,Num Claimants,Injury Type,Overnight Hospital Stay,Claim Amount,Total Claimed,Num Claims,Num Soft Tissue,% Soft Tissue,Claim Amount Received,Fraud Flag
0,1,CI,0,NaN,2,Soft Tissue,No,1625,3250,2,2.0,1.0,0,1
1,2,CI,0,NaN,2,Back,Yes,15028,60112,1,0.0,0.0,15028,0
2,3,CI,54613,Married,1,Broken Limb,No,-99999,0,0,0.0,0.0,572,0
3,4,CI,0,NaN,3,Serious,Yes,270200,0,0,0.0,0.0,270200,0
4,5,CI,0,NaN,4,Soft Tissue,No,8869,0,0,0.0,0.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,496,CI,0,NaN,1,Soft Tissue,No,2118,0,0,0.0,0.0,0,1
496,497,CI,29280,Married,4,Broken Limb,Yes,3199,0,0,NaN,0.0,0,1
497,498,CI,0,NaN,1,Broken Limb,Yes,32469,0,0,0.0,0.0,16763,0
498,499,CI,46683,Married,1,Broken Limb,No,179448,0,0,0.0,0.0,179448,0


In [ ]:
# display all rows in an easy to read format
print(tabulate(df, headers = 'keys', tablefmt = 'grid'))

**Columns & data types in DataFrame**


In [ ]:
df.dtypes

ID                           int64
Insurance Type              object
Income of Policy Holder      int64
Marital Status              object
Num Claimants                int64
Injury Type                 object
Overnight Hospital Stay     object
Claim Amount                 int64
Total Claimed                int64
Num Claims                   int64
Num Soft Tissue            float64
% Soft Tissue              float64
Claim Amount Received        int64
Fraud Flag                   int64
dtype: object

**Display data (Series) of a single column**

In [ ]:
insurance_type_series = df['Insurance Type ']
# insurance_type_series.array
print(tabulate(pd.DataFrame(insurance_type_series.to_list(), columns = ['Insurance Type']), headers='keys', tablefmt='grid'))

**Add new column "Revenue"**

In [ ]:
df['Revenue'] = df["Num Claimants"] + df["Claim Amount Received"]
print(tabulate(pd.DataFrame(df, columns = ["Num Claimants", "Claim Amount Received", "Revenue"]), headers='keys', tablefmt='grid'))

**Select and display a specific row**

In [ ]:
df.iloc[0:1, 0:5]

,ID,Insurance Type,Income of Policy Holder,Marital Status,Num Claimants
0,1,CI,0,NaN,2


### Step 3: Identify missing attribute field(s). Which have missing values and how many? Propose a way to resolve these missing values for those attributes.

**Missing values and count**

In [ ]:
df.isna().sum()

ID                           0
Insurance Type               0
Income of Policy Holder      0
Marital Status             330
Num Claimants                0
Injury Type                  0
Overnight Hospital Stay      0
Claim Amount                 0
Total Claimed                0
Num Claims                   0
Num Soft Tissue             10
% Soft Tissue                0
Claim Amount Received        0
Fraud Flag                   0
dtype: int64

**Propose a way to resolve these missing values**

*Marital Status*

To resolve this, i would suggest creating a new valid value called "Not Specifed", which would replace all the NaN values.

In [ ]:
# Marital Status
# With values
len(df.loc[df['Marital Status'].isin(['Married', 'Single', 'Divorced']), "Marital Status"])

# With missing values
len(df.loc[~df['Marital Status'].isin(['Married', 'Single', 'Divorced']), "Marital Status"])



330

*Num Soft Tissue*

For this attribute i would suggest replacing the NaN values with 0.

In [ ]:
df_nst = df['Num Soft Tissue'].fillna(0, inplace=False)
df_nst

# print(tabulate(pd.DataFrame(df_nst), headers = 'keys', tablefmt = 'grid'))

### Step 4: Consider the attribute "Insurance Type".

**Do you find it odd?**

1. The attribute has the same value in all rows, hence the feature is constant and not important to use the data or solve any possible problem at this point; it's likely not important for our machine learning algorithm

2. The attribute has a space in its name; 'Insurance Type '

In [ ]:
# Values are all the same - 500
print("Count values", df.groupby(['Insurance Type ']).count())
print(df['Insurance Type '])

Count values                   ID  ...  Fraud Flag
Insurance Type        ...            
CI               500  ...         500

[1 rows x 13 columns]
0      CI
1      CI
2      CI
3      CI
4      CI
       ..
495    CI
496    CI
497    CI
498    CI
499    CI
Name: Insurance Type , Length: 500, dtype: object


**How would you remove that attribute**

*Dropping constant attribute*

If the column was numeric, we could use variance threshold to determine if it was constant.

In [ ]:
df_insurance_type_dropped = df.drop('Insurance Type ', axis=1, inplace=False)

print(tabulate(pd.DataFrame(df_insurance_type_dropped), headers = 'keys', tablefmt = 'grid'))

### Step 5: Explore how the attributes vary or relate to each other

**Calculate and visualize correlations using correlation matrix**

In [11]:
corr_matrix = df.corr()
# print(tabulate(df.head(3), headers="keys", tablefmt="grid"))
print(tabulate(corr_matrix, headers="keys", tablefmt="grid"))

+-------------------------+------------+---------------------------+-----------------+----------------+-----------------+--------------+-------------------+-----------------+-------------------------+--------------+
|                         |         ID |   Income of Policy Holder |   Num Claimants |   Claim Amount |   Total Claimed |   Num Claims |   Num Soft Tissue |   % Soft Tissue |   Claim Amount Received |   Fraud Flag |
+=========================+============+===========================+=================+================+=================+==============+===================+=================+=========================+==============+
| ID                      |  1         |                0.041174   |     -0.0877926  |    -0.0177573  |      -0.0829094 |  -0.0777197  |       -0.129436   |      -0.0679916 |             -0.0482813  |    0.0820522 |
+-------------------------+------------+---------------------------+-----------------+----------------+-----------------+--------------+

*Visual representation*

In [ ]:
sn.heatmap(corr_matrix, annot=True)
plt.show()

**Keep and eliminate some attributes based on correlation matrix**

Attributes with no strong or perfect correlation with any other attribute will be dropped. A strong correlation for r is;

- Perfect : 0.9 to 1.0 and -0.9 to -1
- Strong  : 0.5 to 0.9 and -0.5 to -0.9

The attributes to be dropped are;

- ID
- Insurance Type
- Income of Policy Holder
- Marital Status
- Num Soft Tissue
- % Soft Tissue
- Fraud Flag

The attributes retained;

- Num Claimants
- Injury Type
- Overnight Hospital Stay
- Claim Amount
- Total Claimed
- Num Claims
- Claim Amount Received

In [ ]:
df_corr_weak_uncorrelated = df.drop([
                                     'ID',
                                     'Insurance Type ',
                                     'Income of Policy Holder',
                                     'Marital Status',
                                     'Num Soft Tissue',
                                     '% Soft Tissue',
                                     'Fraud Flag'
                                    ], axis=1, inplace=False)
print(tabulate(df_corr_weak_uncorrelated, headers="keys", tablefmt="grid", showindex=False))